In [6]:
import numpy as np
from scipy.special import ellipk
from scipy.constants import epsilon_0, mu_0, pi
import pandas as pd

In [7]:
class ABCD:
    """
    ABCD matrix method
    """

    def __init__(self, freq, length, alpha, beta, couplingCapacitance, charImpedance, loadResistance=50):
        self.freq = freq
        self.length = length
        self.gamma = alpha + 1j*beta
        self.couplingCapacitance = couplingCapacitance
        self.charImpedance = charImpedance
        self.loadResistance = loadResistance
        
        self.s21 = self.s21(self.abcd(
            self.input(self.freq, self.couplingCapacitance),
            self.transmission(self.freq, self.length, self.gamma, self.charImpedance),
            self.output(self.freq, self.couplingCapacitance)),
            self.loadResistance)

    def input(self, freq, couplingCapacitance):
        n=np.size(freq)
        Zin = 1/(1j*freq*couplingCapacitance)
        return np.append(np.ones(n),[np.zeros(n),Zin,np.ones(n)]).reshape(n,2,2,order='F')
        
    def output(self, freq, couplingCapacitance):
        n=np.size(freq)
        Zin = 1/(1j*freq*couplingCapacitance)
        return np.append(np.ones(n),[np.zeros(n),Zin,np.ones(n)]).reshape(n,2,2,order='F')
        
    def transmission(self, freq, length, gamma, charImpedance):
        n=np.size(freq)
        t11 = np.cosh(gamma * length)
        t12 = charImpedance * np.sinh(gamma * length)
        t21 = (1/charImpedance) * np.sinh(gamma * length)
        t22 = np.cosh(gamma * length)
        return np.append(t11*np.ones(n),[t21*np.ones(n),t12*np.ones(n),t22*np.ones(n)]).reshape(n,2,2,order='F')
    
    def abcd(self, input, transmission, output):
        return input*transmission*output
    
    def s21(self, pABCD, loadResistance):
        A = pABCD[:,0,0]
        B = pABCD[:,0,1]
        C = pABCD[:,1,0]
        D = pABCD[:,1,1]
        RL = loadResistance
        return 2/( A + (B/RL)+ (C*RL) + D )

In [8]:
class Conductor:
    """
    CPW conductor material class
    """
    
    materials = {'Niobium': {
                     'superconductor':                    True,
                     'criticalTemperature':               9.2,
                     'londonPenetrationDepthZero':        33.3E-9,
                     'resistancePerUnitLength':           0,
                     'conductancePerUnitLength':          0,
                     'amplitudeAttenuationPerUnitLength': 2.4E-4},
                 'Niobium Nitride': {
                     'superconductor':                    True,
                     'criticalTemperature':               16.2,
                     'londonPenetrationDepthZero':        40.0E-9,
                     'resistancePerUnitLength':           0,
                     'conductancePerUnitLength':          0,
                     'amplitudeAttenuationPerUnitLength': 2.4E-4},
                'Copper': {
                     'superconductor':                    False}}

    def __init__(self, material):
        self.material                          = material
        self.superconductor                    = self.materials[material]['superconductor']
        if self.superconductor:
            self.criticalTemperature               = self.materials[material]['criticalTemperature']
            self.londonPenetrationDepthZero        = self.materials[material]['londonPenetrationDepthZero']
            self.resistancePerUnitLength           = self.materials[material]['resistancePerUnitLength']
            self.conductancePerUnitLength          = self.materials[material]['conductancePerUnitLength']
            self.amplitudeAttenuationPerUnitLength = self.materials[material]['amplitudeAttenuationPerUnitLength']
    
    def info():
        return pd.DataFrame(Conductor.materials)

class Substrate:
    """
    Substrate material class
    """
    
    materials = {'Silicon': {
                     'relativePermittivity': 11.9},
                 'Sapphire': {
                     'relativePermittivity': 10.2},
                'ArlonAD1000': {
                     'relativePermittivity': 10.2}}

    def __init__(self, material):
        self.material = material
        self.relativePermittivity = self.materials[material]['relativePermittivity']
        
    def info():
        return pd.DataFrame(Substrate.materials)

In [13]:
class CPWResonator:
    """
    Coplanar wave-guide resonator class
    """

    def __init__(self, length, conductorWidth, gapWidth, conductorThickness, resonatorType, 
                 conductorMaterial, substrateMaterial,
                 temperature=4, couplingCapacitance=0, loadImpedance=50, loadBoundaryCondition='Short', mode=1):
        # Supplied parameters
        self.length = np.array(length)
        self.conductorWidth = np.array(conductorWidth)
        self.gapWidth = np.array(gapWidth)
        self.conductorThickness = np.array(conductorThickness)
        self.resonatorType = np.array(resonatorType)
        self.conductor = Conductor(conductorMaterial)
        self.substrate = Substrate(substrateMaterial)
        self.temperature = np.array(temperature)
        self.couplingCapacitance = np.array(couplingCapacitance)
        self.loadImpedance = np.array(loadImpedance)
        self.loadBoundaryCondition = np.array(loadBoundaryCondition)
        self.mode = np.array(mode)

    def effectivePermittivity(self):
        return (1 + self.substrate.relativePermittivity)/2

    def capacitancePerUnitLength(self):
        # Complete elliptic integral of the first kind
        k = self.conductorWidth / (self.conductorWidth + 2 * self.gapWidth)
        k2 = np.sqrt(1 - k**2)

        # Total CPW capacitance p.u.l.
        return 4 * epsilon_0 * (self.effectivePermittivity() + 0) * (ellipk(k) / ellipk(k2))
    
    def totalInductancePerUnitLength(self):
        if self.conductor.superconductor:
            return self.geometricInductancePerUnitLength() + self.kineticInductancePerUnitLength()
        else:
            return self.geometricInductancePerUnitLength()

    def geometricInductancePerUnitLength(self):
        # Complete elliptic integral of the first kind
        k = self.conductorWidth / (self.conductorWidth + 2 * self.gapWidth)
        k2 = np.sqrt(1 - k**2)

        # Total conductor geometric inductance p.u.l.
        return (mu_0 / 4) * (ellipk(k2) / ellipk(k))

    def kineticInductancePerUnitLength(self):
        # Complete elliptic integral of the first kind
        k = self.conductorWidth / (self.conductorWidth + 2 * self.gapWidth)
        K = ellipk(k)

        # Geometrical factor
        s, W, T = self.gapWidth, self.conductorWidth, self.conductorThickness
        geometricFactor = (1 / (2 * k**2 * K**2)) * (- np.log(T / (4 * W)) + ((2 * (W + s))
            / (W + 2 * s)) * np.log(s / (W + s)) - (W / (W + 2 * s)) * np.log(T / (4 * (W + 2 * s))))

        # Kinetic Inductance p.u.l.
        return mu_0 * (self.londonPenetrationDepthT()**2 / (W * T)) * geometricFactor


    def londonPenetrationDepthT(self):
        return self.conductor.londonPenetrationDepthZero /                np.sqrt(1 - (self.temperature / self.conductor.criticalTemperature)**4)

    def characteristicImpedance(self, resistance=0, conductance=0, frequency=1):
        return np.sqrt(
            (resistance  + 1j*2 * pi * frequency * self.totalInductancePerUnitLength() ) /
            (conductance + 1j*2 * pi * frequency * self.capacitancePerUnitLength()))

    def inputImpedance(self):
        gamma = np.sqrt(
            (self.conductor.resistancePerUnitLength + 
             1j*2*pi*self.coupledResonantFrequency()*self.totalInductancePerUnitLength() ) *
            (self.conductor.conductancePerUnitLength + 
             1j*2*pi*self.coupledResonantFrequency()*self.capacitancePerUnitLength()))

        if self.loadBoundaryCondition == 'Short':
            return self.characteristicImpedance() * np.tanh(gamma * self.length)
        elif self.loadBoundaryCondition == 'Open':
            return self.characteristicImpedance() / np.tanh(gamma * self.length)
        else:
            print('Error: Load boundary condition no valid!')
            return -1

    def uncoupledResonantFrequency(self):
        m = self.getModeFactor()
        return 1 / (np.sqrt(self.totalInductancePerUnitLength()*self.capacitancePerUnitLength()) * m * self.length)

    def coupledResonantFrequency(self):
        m = self.getModeFactor()        
        return 1 / (np.sqrt((self.totalInductancePerUnitLength() * self.length) * (
            (self.capacitancePerUnitLength() * self.length) + self.effectiveCouplingCapacitance())) * m)

    def effectiveCouplingCapacitance(self):
        return self.couplingCapacitance / (1 + 
               (self.uncoupledResonantFrequency() * self.couplingCapacitance * self.loadImpedance * pi)**2)

    def internalQualityFactor(self):
        m = self.getModeFactor()
        return (1/m) * (pi/(self.conductor.amplitudeAttenuationPerUnitLength * self.length))
    
    def externalQualityFactor(self, method=0):
        if method == 0:
            return self.externalQualityFactorMain()
        elif method == 1:
            return self.externalQualityFactorApprox()
        elif method == 2:
            return self.externalQualityFactorQWref()
    
    def externalQualityFactorMain(self, loadResistance=50):
        omega_n = 2 * pi * self.uncoupledResonantFrequency()
        r_star = (1+(omega_n*self.couplingCapacitance*loadResistance)**2) / ((omega_n*self.couplingCapacitance)**2 * loadResistance)
        C = (self.capacitancePerUnitLength() * self.length)/2
        return omega_n * r_star * C

    def externalQualityFactorApprox(self):
        m = self.getModeFactor()
        q_in = 2 * pi * self.uncoupledResonantFrequency() * self.couplingCapacitance * self.characteristicImpedance()
        return (1/m) * (pi/(q_in**2))

    def externalQualityFactorQWref(self, inputPortImpedance = 50):
        m = self.getModeFactor()
        omega_0 = 2 * pi * self.uncoupledResonantFrequency()
        mBody = 1/(omega_0**2 * self.couplingCapacitance**2 * self.characteristicImpedance() * inputPortImpedance)
        return (pi/m) * mBody
    
    def loadedQualityFactor(self):
        return 1 / ( (1/self.internalQualityFactor()) + (1/self.externalQualityFactor()) )

    def getModeFactor(self):
        if self.resonatorType == 'half':
            m = 4.0 / (2.0 * self.mode)
        elif self.resonatorType == 'quarter':
            m = 4.0 / ((2.0 * self.mode) - 1)
        else:
            print('Error: Incorrect resonator type provided!')
            return -1
        return m

    def insertionLoss(self):
        g = self.internalQualityFactor()/self.externalQualityFactor()
        return -20 * np.log10(g/(g+1))
    
    def beta(self):
        omega_n = 2 * pi * self.uncoupledResonantFrequency()
        return omega_n * np.sqrt(self.totalInductancePerUnitLength()*self.capacitancePerUnitLength())
    
    def info(self):
        supplied_parameters = {
            'length': self.length,
            'conductorWidth': self.conductorWidth,
            'gapWidth': self.gapWidth,
            'conductorThickness': self.conductorThickness,
            'resonatorType': [self.resonatorType],
            'conductor': [self.conductor.material],
            'substrate': [self.substrate.material],
            'temperature': self.temperature,
            'couplingCapacitance': self.couplingCapacitance,
            'loadImpedance': self.loadImpedance,
            'loadBoundaryCondition': [self.loadBoundaryCondition],
            'mode': self.mode
        }
        calculated_parameters = {
            'effectivePermittivity': self.effectivePermittivity(),
            'capacitancePerUnitLength': self.capacitancePerUnitLength(),
            'totalInductancePerUnitLength': self.totalInductancePerUnitLength(),
            'geometricInductancePerUnitLength': self.geometricInductancePerUnitLength(),
            'kineticInductancePerUnitLength': self.kineticInductancePerUnitLength(),
            'londonPenetrationDepthT': self.londonPenetrationDepthT(),
            'characteristicImpedance': self.characteristicImpedance(),
            'inputImpedance': self.inputImpedance(),
            'uncoupledResonantFrequency': self.uncoupledResonantFrequency(),
            'coupledResonantFrequency': self.coupledResonantFrequency(),
            'effectiveCouplingCapacitance': self.effectiveCouplingCapacitance(),
            'internalQualityFactor': self.internalQualityFactor(),
            'externalQualityFactor': self.externalQualityFactor(),
            'loadedQualityFactor': self.loadedQualityFactor(),
            'insertionLoss': self.insertionLoss(),
            'beta': self.beta()
        }
        return [pd.DataFrame.transpose(pd.DataFrame(supplied_parameters, index=['Supplied parameters'])),
                pd.DataFrame.transpose(pd.DataFrame(calculated_parameters, index=['Calculated parameters']))]

In [10]:
import matplotlib.pyplot as plt 
from scipy.constants import c
%matplotlib inline

In [11]:
mCPW = CPWResonator(length                = [7153E-6],
                    conductorWidth        = [30E-3],
                    gapWidth              = [19E-3],
                    conductorThickness    = [1E-3],
                    resonatorType         = 'quarter',
                    conductorMaterial     = 'Copper',
                    substrateMaterial     = 'ArlonAD1000',
                    temperature           = [3],
                    couplingCapacitance   = [10E-15],
                    loadBoundaryCondition = 'Short',
                    mode                  = [3])

mCPW.characteristicImpedance()

array([56.5124598+0.j])

In [14]:
default = {'length':                [7153E-6],
           'conductorWidth':        [20E-6],
           'gapWidth':              [10E-6],
           'conductorThickness':    [100E-9],
           'resonatorType':         'quarter',
           'conductorMaterial':     'Niobium Nitride',
           'substrateMaterial':     'Silicon',
           'temperature':           [3],
           'couplingCapacitance':   [10E-15],
           'loadBoundaryCondition': 'Short',
           'mode':                  [1]}